In [149]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

In [150]:
key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [151]:
load_dotenv()

True

In [152]:
from langchain_community.llms import HuggingFaceHub

In [153]:
key = os.getenv('HUGGINGFACEHUB_API_KEY')

In [154]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [155]:
print(llm)

model='gemini-pro' client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)


In [156]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [157]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [158]:
TEMPLATE="""
Text: {text}
You are an expert MCQ Maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

{response_json}
"""

In [159]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [160]:
quiz_chain = LLMChain(llm = llm,
                      prompt = quiz_generation_prompt,
                      output_key='quiz',
                      verbose = True)

In [161]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [162]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template=TEMPLATE2
)

In [163]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, 
                      output_key="review", verbose=True)

In [164]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], 
                                        input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [165]:
file_path = r'C:\Users\Manan Bedi\Documents\LLM_iNeuron\MCQ_Generator\data.txt'

In [166]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [167]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [168]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [169]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [170]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defin


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
RESPONSE_JSON = {
"1": {"mcq": "What is a unifying theme that ties all organisms together?", "options": {"a": "All organisms are made of cells.", "b": "All organisms need water.", "c": "All organisms reproduce.", "d": "All organisms have a nervous system."}, "correct": "a"},
"2": {"mcq": "Which of the following is NOT a subdiscipline of biology?", "options": {"a": "Molecular biology", "b": "Physics", "c": "Anatomy", "d": "Evolution"}, "correct": "b"},
"

In [171]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [172]:
quiz=response.get("quiz")

In [177]:
quiz[15:]

' {\n"1": {"mcq": "What is a unifying theme that ties all organisms together?", "options": {"a": "All organisms are made of cells.", "b": "All organisms need water.", "c": "All organisms reproduce.", "d": "All organisms have a nervous system."}, "correct": "a"},\n"2": {"mcq": "Which of the following is NOT a subdiscipline of biology?", "options": {"a": "Molecular biology", "b": "Physics", "c": "Anatomy", "d": "Evolution"}, "correct": "b"},\n"3": {"mcq": "What is an important aspect of life that allows organisms to grow and move?", "options": {"a": "Energy processing", "b": "Oxygen consumption", "c": "Nutrient absorption", "d": "Waste elimination"}, "correct": "a"},\n"4": {"mcq": "How do biologists study life?", "options": {"a": "Through observation and experimentation", "b": "By reading books and articles", "c": "Through intuition and guesswork", "d": "By listening to what other people say"}, "correct": "a"},\n"5": {"mcq": "What is the scientific method?", "options": {"a": "A process f

In [178]:
quiz=json.loads(quiz[15:])

In [179]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [180]:
quiz_table_data

[{'MCQ': 'What is a unifying theme that ties all organisms together?',
  'Choices': 'a: All organisms are made of cells. | b: All organisms need water. | c: All organisms reproduce. | d: All organisms have a nervous system.',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is NOT a subdiscipline of biology?',
  'Choices': 'a: Molecular biology | b: Physics | c: Anatomy | d: Evolution',
  'Correct': 'b'},
 {'MCQ': 'What is an important aspect of life that allows organisms to grow and move?',
  'Choices': 'a: Energy processing | b: Oxygen consumption | c: Nutrient absorption | d: Waste elimination',
  'Correct': 'a'},
 {'MCQ': 'How do biologists study life?',
  'Choices': 'a: Through observation and experimentation | b: By reading books and articles | c: Through intuition and guesswork | d: By listening to what other people say',
  'Correct': 'a'},
 {'MCQ': 'What is the scientific method?',
  'Choices': 'a: A process for acquiring knowledge through observation and experimentation | b

In [181]:
quiz=pd.DataFrame(quiz_table_data)

In [182]:
quiz.to_csv("mcq.csv",index=False)

In [183]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'04_07_2024_16_00_24'